# 🚀 POC: Busca Vetorial OTIMIZADA para AWS g4dn.2xlarge

## 🎯 Instância: g4dn.2xlarge
- **GPU**: NVIDIA T4 (16GB VRAM) - Tensor Cores
- **vCPUs**: 8 cores
- **RAM**: 32GB
- **Custo**: ~$0.75/h (on-demand) | ~$0.25/h (spot)

## 🚀 Otimizações para T4:

### 1. **Modelo de embedding otimizado:**
- ✅ `paraphrase-multilingual-MiniLM-L12-v2` (384 dims, 4x mais rápido)
- ✅ **Mixed Precision (FP16)**: 2x speedup nos Tensor Cores da T4
- ✅ **Batch size: 256** (aproveita 16GB VRAM vs 32 na CPU)

### 2. **FAISS IndexHNSWFlat:**
- ✅ Sem treinamento (ao contrário de IVF)
- ✅ **99.5% de precisão** (vs 95-98% IVF)
- ✅ **10-50x mais rápido** que Flat
- ⚡ Opcional: FAISS GPU (5-10x mais rápido na busca)

### 3. **Performance esperada:**
| Métrica | CPU | GPU T4 |
|---------|-----|--------|
| Construção | 30-40min | **5-10min** |
| Busca | 80-100ms | **30-50ms** |
| Throughput | ~100 q/s | **~1000 q/s** |

---

## 📦 Setup e GPU Detection

In [ ]:
import pandas as pd
import numpy as np
import json
import re
import faiss
import pickle
import torch
from pathlib import Path
from typing import Dict, List, Optional
from sentence_transformers import SentenceTransformer
from unidecode import unidecode
from tqdm import tqdm
import time

print("="*60)
print("🔍 DETECÇÃO DE HARDWARE")
print("="*60)

# Detectar GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"\n🎮 GPU detectada: {gpu_name}")
    print(f"💾 VRAM total: {gpu_memory:.1f}GB")
    print(f"✅ CUDA version: {torch.version.cuda}")
    
    # Verificar se é T4
    if 'T4' in gpu_name:
        print(f"🚀 GPU T4 detectada - Tensor Cores disponíveis!")
        print(f"⚡ Mixed Precision (FP16) será ativado automaticamente")
    
    # FAISS GPU disponível?
    try:
        res = faiss.StandardGpuResources()
        faiss_gpu_available = True
        print(f"✅ FAISS GPU disponível (busca 5-10x mais rápida)")
    except:
        faiss_gpu_available = False
        print(f"⚠️  FAISS CPU only (para GPU: pip install faiss-gpu)")
else:
    print(f"\n⚠️  Executando em CPU")
    print(f"💡 Recomendado: usar instância g4dn.2xlarge com GPU T4")
    faiss_gpu_available = False

print(f"\n🔧 Device para embeddings: {device}")
print("="*60)

## 1. EmbeddingService - Otimizado para T4

In [ ]:
class EmbeddingServiceGPU:
    """Serviço otimizado para GPU NVIDIA T4 (g4dn.2xlarge)"""
    
    def __init__(
        self, 
        model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        use_fp16: bool = True
    ):
        """
        Modelos testados na T4 (velocidade vs qualidade):
        
        RÁPIDO (recomendado para produção):
        - paraphrase-multilingual-MiniLM-L12-v2 (384 dims) ✅ MELHOR CUSTO-BENEFÍCIO
        - all-MiniLM-L6-v2 (384 dims, inglês mas OK em PT)
        
        PRECISO (se precisar mais qualidade):
        - paraphrase-multilingual-mpnet-base-v2 (768 dims, 2x mais lento)
        - neuralmind/bert-base-portuguese-cased (768 dims, 3x mais lento)
        
        Args:
            use_fp16: Mixed precision (FP16) - 2x speedup na T4 (Tensor Cores)
        """
        print(f"⚡ Carregando modelo: {model_name}")
        self.model = SentenceTransformer(model_name, device=device)
        
        # Mixed precision para T4 (Tensor Cores)
        if device == 'cuda' and use_fp16:
            self.model.half()  # Converte para FP16
            print(f"🚀 Mixed Precision (FP16) ATIVADO - 2x speedup")
        
        self.embedding_dim = self.model.get_sentence_embedding_dimension()
        print(f"✅ Embedding dimension: {self.embedding_dim}")
        print(f"🔧 Device: {self.model.device}")
        
        # Batch size otimizado para T4 (16GB VRAM)
        if device == 'cuda':
            # T4 aguenta batch 256 com MiniLM (384 dims)
            # Se usar modelo maior (768 dims), reduza para 128
            self.optimal_batch_size = 256 if self.embedding_dim <= 384 else 128
        else:
            self.optimal_batch_size = 32
        
        print(f"📦 Batch size otimizado: {self.optimal_batch_size}")
    
    @staticmethod
    def normalize_text(text: str) -> str:
        """Normalização de endereços brasileiros"""
        if not text or not isinstance(text, str):
            return ""
        
        text = unidecode(text)
        text = text.lower()
        
        # Expandir abreviações (aceita com/sem ponto)
        replacements = {
            r'\br\.?\s': 'rua ',
            r'\bav\.?\s': 'avenida ',
            r'\btrav\.?\s': 'travessa ',
            r'\balam\.?\s': 'alameda ',
            r'\bpca\.?\s': 'praca ',
            r'\bjd\.?\s': 'jardim ',
            r'\bvl\.?\s': 'vila ',
            r'\bcj\.?\s': 'conjunto ',
            r'\bqd\.?\s': 'quadra ',
            r'\blt\.?\s': 'lote ',
        }
        
        for pattern, replacement in replacements.items():
            text = re.sub(pattern, replacement, text)
        
        # Remover pontuação e múltiplos espaços
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    def embed_text(self, text: str) -> np.ndarray:
        """Embedding de um texto (usado na busca)"""
        normalized = self.normalize_text(text)
        if not normalized:
            return np.zeros(self.embedding_dim, dtype=np.float32)
        return self.model.encode(
            normalized, 
            convert_to_numpy=True, 
            show_progress_bar=False,
            normalize_embeddings=True  # ← CRÍTICO: deve estar igual ao embed_batch!
        )
    
    def embed_address_fields(self, address: Dict[str, str]) -> Dict[str, np.ndarray]:
        """Embedding de múltiplos campos de um endereço"""
        embeddings = {}
        for field in ['logradouro', 'bairro', 'cidade']:
            text = address.get(field, '')
            embeddings[field] = self.embed_text(text)
        return embeddings
    
    def embed_batch(self, texts, batch_size: int = None) -> np.ndarray:
        """
        Embedding de batch otimizado para T4:
        - CPU: batch=32 (limitado por RAM)
        - GPU T4: batch=256 (16GB VRAM + FP16)
        """
        if batch_size is None:
            batch_size = self.optimal_batch_size
        
        # Converter Series para lista se necessário
        if hasattr(texts, 'tolist'):
            texts = texts.tolist()
        
        # Normalizar todos os textos
        normalized_texts = [self.normalize_text(t) for t in texts]
        normalized_texts = [t if t else " " for t in normalized_texts]
        
        # Encode com configurações otimizadas
        embeddings = self.model.encode(
            normalized_texts,
            convert_to_numpy=True,
            show_progress_bar=True,
            batch_size=batch_size,
            device=device,
            normalize_embeddings=True  # L2 normalization (melhora similaridade)
        )
        
        return embeddings.astype(np.float32)

## 2. IndexBuilder - HNSW com suporte GPU

In [ ]:
class IndexBuilderGPU:
    """Construtor de índices FAISS otimizado para T4"""
    
    def __init__(self, embedding_service: EmbeddingServiceGPU, use_gpu_index: bool = False):
        """
        Args:
            use_gpu_index: Transferir índices para GPU (busca 5-10x mais rápida)
                          Requer: pip install faiss-gpu
                          Atenção: consome VRAM (pode conflitar com embeddings)
        """
        self.embedding_service = embedding_service
        self.indices = {}
        self.dataframe = None
        self.use_gpu_index = use_gpu_index and faiss_gpu_available
        
        if self.use_gpu_index:
            self.gpu_resource = faiss.StandardGpuResources()
            # Reservar 8GB para índices (deixa 8GB para embeddings)
            self.gpu_resource.setTempMemory(8 * 1024 * 1024 * 1024)
            print(f"🎮 FAISS GPU ativado (8GB reservados para índices)")
    
    def build_indices(
        self,
        df: pd.DataFrame,
        fields: list = None,
        use_hnsw: bool = True,
        M: int = 32,
        efSearch: int = 32
    ) -> dict:
        """
        Constrói índices FAISS HNSW otimizados
        
        Args:
            use_hnsw: True=HNSW (rápido+preciso), False=Flat (lento)
            M: Conexões no grafo HNSW (16-64)
               - M=16: rápido, ~98% recall
               - M=32: balanceado, ~99.5% recall ✅
               - M=64: preciso, ~99.9% recall, +memória
            efSearch: Vizinhos na busca (16-64)
               - 16: muito rápido
               - 32: balanceado ✅
               - 64: mais preciso
        """
        if fields is None:
            fields = ['logradouro', 'bairro', 'cidade']
        
        self.dataframe = df.copy()
        n_records = len(df)
        
        print(f"\n{'='*60}")
        print(f"🔨 CONSTRUINDO ÍNDICES FAISS")
        print(f"{'='*60}")
        print(f"📊 Total de registros: {n_records:,}")
        print(f"⚙️  Modo: {'HNSW (rápido+preciso)' if use_hnsw else 'Flat (lento)'}")
        print(f"🎯 Parâmetros: M={M}, efSearch={efSearch}")
        print(f"")
        
        total_start = time.time()
        
        for field in fields:
            print(f"\n📍 Campo: {field}")
            print(f"{'-'*40}")
            field_start = time.time()
            
            # Embedding do campo
            texts = df[field].fillna('').astype(str).tolist()
            print(f"   ⚡ Gerando embeddings...")
            embeddings = self.embedding_service.embed_batch(texts)
            dimension = embeddings.shape[1]
            
            # Criar índice
            if use_hnsw:
                print(f"   🧠 Criando índice HNSW (dim={dimension})...")
                index_cpu = faiss.IndexHNSWFlat(dimension, M)
                index_cpu.hnsw.efSearch = efSearch
                
                print(f"   📥 Adicionando {n_records:,} vetores...")
                index_cpu.add(embeddings)
                
                # GPU (opcional)
                if self.use_gpu_index:
                    print(f"   🎮 Transferindo para GPU T4...")
                    try:
                        index = faiss.index_cpu_to_gpu(self.gpu_resource, 0, index_cpu)
                        print(f"   ✅ Índice na GPU")
                    except Exception as e:
                        print(f"   ⚠️  Falha GPU: {e}")
                        print(f"   ℹ️  Usando índice CPU")
                        index = index_cpu
                else:
                    index = index_cpu
            else:
                print(f"   🧠 Criando índice Flat (dim={dimension})...")
                index = faiss.IndexFlatL2(dimension)
                index.add(embeddings)
                
                if self.use_gpu_index:
                    index = faiss.index_cpu_to_gpu(self.gpu_resource, 0, index)
            
            self.indices[field] = index
            
            elapsed = time.time() - field_start
            print(f"   ✅ Concluído em {elapsed:.1f}s")
        
        total_elapsed = time.time() - total_start
        print(f"\n{'='*60}")
        print(f"🎉 TODOS OS ÍNDICES CONSTRUÍDOS")
        print(f"⏱️  Tempo total: {total_elapsed/60:.1f}min")
        print(f"{'='*60}\n")
        
        return self.indices
    
    def save_indices(self, output_dir: str):
        """Salva índices para reutilização"""
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        print(f"\n💾 Salvando índices em: {output_path}")
        
        for field, index in self.indices.items():
            # Se índice está na GPU, transferir para CPU antes de salvar
            if self.use_gpu_index:
                index_cpu = faiss.index_gpu_to_cpu(index)
            else:
                index_cpu = index
            
            index_file = output_path / f"{field}_index.faiss"
            faiss.write_index(index_cpu, str(index_file))
            print(f"   ✅ {field}_index.faiss")
        
        # Salvar dataframe
        df_file = output_path / "addresses.parquet"
        self.dataframe.to_parquet(df_file, index=False)
        print(f"   ✅ addresses.parquet")
        
        # Metadata
        metadata = {
            'fields': list(self.indices.keys()),
            'n_records': len(self.dataframe),
            'embedding_dim': self.embedding_service.embedding_dim
        }
        metadata_file = output_path / "metadata.pkl"
        with open(metadata_file, 'wb') as f:
            pickle.dump(metadata, f)
        print(f"   ✅ metadata.pkl")
        
        print(f"\n🎉 Índices salvos! Use load_indices() para carregar.")
    
    def load_indices(self, input_dir: str):
        """Carrega índices salvos (MUITO mais rápido)"""
        input_path = Path(input_dir)
        
        print(f"\n📂 Carregando índices de: {input_path}")
        start_time = time.time()
        
        # Metadata
        metadata_file = input_path / "metadata.pkl"
        with open(metadata_file, 'rb') as f:
            metadata = pickle.load(f)
        
        # DataFrame
        df_file = input_path / "addresses.parquet"
        self.dataframe = pd.read_parquet(df_file)
        
        # Índices
        for field in metadata['fields']:
            index_file = input_path / f"{field}_index.faiss"
            index_cpu = faiss.read_index(str(index_file))
            
            # Transferir para GPU se ativado
            if self.use_gpu_index:
                try:
                    index = faiss.index_cpu_to_gpu(self.gpu_resource, 0, index_cpu)
                    print(f"   ✅ {field} (GPU)")
                except:
                    index = index_cpu
                    print(f"   ✅ {field} (CPU)")
            else:
                index = index_cpu
                print(f"   ✅ {field}")
            
            self.indices[field] = index
        
        elapsed = time.time() - start_time
        print(f"\n⚡ Carregado em {elapsed:.1f}s ({len(self.dataframe):,} registros)\n")
        
        return self.indices, self.dataframe

## 3. SearchEngine (mesmo da versão anterior)

In [ ]:
class SearchEngine:
    """Motor de busca vetorial com pesos dinâmicos"""
    
    def __init__(
        self,
        embedding_service: EmbeddingServiceGPU,
        indices: Dict[str, faiss.Index],
        dataframe: pd.DataFrame
    ):
        self.embedding_service = embedding_service
        self.indices = indices
        self.dataframe = dataframe
        
        self.base_weights = {
            'with_cep': {
                'cep': 0.30,
                'logradouro': 0.40,
                'bairro': 0.20,
                'cidade': 0.10
            },
            'without_cep': {
                'logradouro': 0.55,
                'bairro': 0.25,
                'cidade': 0.20
            }
        }
        
        self.use_uf_filter = True
        self.confidence_threshold = 0.8
    
    def _get_dynamic_weights(self, query: Dict[str, str]) -> Dict[str, float]:
        has_cep = bool(query.get('cep'))
        weights = self.base_weights['with_cep' if has_cep else 'without_cep'].copy()
        available_fields = [f for f in ['logradouro', 'bairro', 'cidade'] if query.get(f)]
        filtered_weights = {k: v for k, v in weights.items() if k in available_fields or k == 'cep'}
        total_weight = sum(filtered_weights.values())
        if total_weight > 0:
            normalized_weights = {k: v / total_weight for k, v in filtered_weights.items()}
        else:
            normalized_weights = filtered_weights
        return normalized_weights
    
    def _calculate_field_similarity(
        self,
        field: str,
        query_embedding: np.ndarray,
        top_k: int = 100
    ) -> tuple:
        index = self.indices[field]
        query_embedding = query_embedding.reshape(1, -1).astype(np.float32)
        distances, indices = index.search(query_embedding, top_k)
        similarities = 1.0 / (1.0 + distances[0])
        return similarities, indices[0]
    
    def _calculate_cep_match(self, query_cep: str, db_cep: str) -> float:
        if not query_cep or not db_cep:
            return 0.0
        
        query_clean = query_cep.replace('-', '').replace('.', '')
        db_clean = db_cep.replace('-', '').replace('.', '')
        
        if query_clean == db_clean:
            return 1.0
        
        if len(query_clean) >= 5 and len(db_clean) >= 5:
            if query_clean[:5] == db_clean[:5]:
                return 0.5
        
        return 0.0
    
    def search(
        self,
        query: Dict[str, str],
        top_k: int = 5,
        search_k: int = 100
    ) -> str:
        weights = self._get_dynamic_weights(query)
        query_embeddings = self.embedding_service.embed_address_fields(query)
        
        candidate_scores = {}
        field_scores_map = {}
        
        for field in ['logradouro', 'bairro', 'cidade']:
            if not query.get(field):
                continue
            
            query_emb = query_embeddings[field]
            similarities, indices = self._calculate_field_similarity(field, query_emb, search_k)
            weight = weights.get(field, 0.0)
            
            for idx, sim in zip(indices, similarities):
                if self.use_uf_filter and query.get('uf'):
                    db_uf = self.dataframe.iloc[idx]['uf']
                    if db_uf != query['uf']:
                        continue
                
                if idx not in candidate_scores:
                    candidate_scores[idx] = 0.0
                    field_scores_map[idx] = {}
                
                candidate_scores[idx] += weight * sim
                field_scores_map[idx][field] = float(sim)
        
        if query.get('cep'):
            cep_weight = weights.get('cep', 0.0)
            for idx in candidate_scores.keys():
                db_cep = self.dataframe.iloc[idx]['cep']
                cep_score = self._calculate_cep_match(query.get('cep'), db_cep)
                candidate_scores[idx] += cep_weight * cep_score
                field_scores_map[idx]['cep'] = cep_score
        
        sorted_candidates = sorted(
            candidate_scores.items(),
            key=lambda x: x[1],
            reverse=True
        )[:top_k]
        
        results = []
        for idx, score in sorted_candidates:
            row = self.dataframe.iloc[idx]
            
            if score >= self.confidence_threshold:
                confidence = "high"
            elif score >= 0.6:
                confidence = "medium"
            else:
                confidence = "low"
            
            result = {
                "address": {
                    "logradouro": row['logradouro'],
                    "bairro": row['bairro'],
                    "cidade": row['cidade'],
                    "uf": row['uf'],
                    "cep": row['cep']
                },
                "score": float(score),
                "confidence": confidence,
                "field_scores": field_scores_map.get(idx, {})
            }
            results.append(result)
        
        response = {
            "results": results,
            "query": query,
            "total_found": len(results),
            "weights_used": weights
        }
        
        return json.dumps(response, ensure_ascii=False, indent=2)

## 3.1. SearchEngineOptimized - Com Filtro de UF Otimizado

In [ ]:
class SearchEngineOptimized(SearchEngine):
    """
    Motor de busca OTIMIZADO que filtra por UF ANTES da busca vetorial.
    
    Resolve o problema de candidatos de estados diferentes serem retornados
    quando existem nomes de ruas/bairros/cidades duplicados em vários estados.
    """
    
    def __init__(
        self,
        embedding_service: EmbeddingServiceGPU,
        indices: Dict[str, faiss.Index],
        dataframe: pd.DataFrame
    ):
        super().__init__(embedding_service, indices, dataframe)
        
        # Criar mapeamento de índices por UF para busca rápida
        print("🗺️  Criando mapeamento de índices por UF...")
        self.uf_to_indices = {}
        for uf in self.dataframe['uf'].unique():
            uf_mask = self.dataframe['uf'] == uf
            self.uf_to_indices[uf] = np.where(uf_mask)[0]
        
        total_ufs = len(self.uf_to_indices)
        print(f"✅ Mapeamento criado: {total_ufs} UFs indexadas")
    
    def _calculate_field_similarity(
        self,
        field: str,
        query_embedding: np.ndarray,
        top_k: int = 100
    ) -> tuple:
        """Busca vetorial com conversão correta de distância para similaridade"""
        index = self.indices[field]
        query_embedding = query_embedding.reshape(1, -1).astype(np.float32)
        distances, indices = index.search(query_embedding, top_k)
        
        # Converter distâncias L2² para similaridade cosseno
        # Para vetores normalizados: ||a-b||² = 2(1 - cos(a,b))
        # Portanto: cos(a,b) = 1 - (||a-b||² / 2)
        similarities = np.clip(1.0 - (distances[0] / 2.0), 0.0, 1.0)
        
        # Considerar distâncias muito pequenas como match perfeito
        epsilon = 1e-6
        similarities[distances[0] < epsilon] = 1.0
        
        return similarities, indices[0]
    
    def search(
        self,
        query: Dict[str, str],
        top_k: int = 5,
        search_k: int = 100
    ) -> str:
        """
        Busca otimizada com filtro de UF aplicado ANTES da busca vetorial.
        
        Estratégia:
        1. Se UF fornecida: busca apenas nos índices daquele estado
        2. Sem UF: busca global (comportamento original)
        """
        weights = self._get_dynamic_weights(query)
        query_embeddings = self.embedding_service.embed_address_fields(query)
        
        # Determinar o conjunto de índices válidos (filtro de UF)
        query_uf = query.get('uf')
        if query_uf and query_uf in self.uf_to_indices:
            valid_indices_set = set(self.uf_to_indices[query_uf])
            print(f"🔍 Filtrando por UF={query_uf}: {len(valid_indices_set):,} registros")
        else:
            valid_indices_set = None  # Busca global
        
        candidate_scores = {}
        field_scores_map = {}
        
        # Busca vetorial por campo
        for field in ['logradouro', 'bairro', 'cidade']:
            if not query.get(field):
                continue
            
            query_emb = query_embeddings[field]
            
            # Buscar mais candidatos para garantir que encontramos matches do UF correto
            search_k_adjusted = search_k * 10 if valid_indices_set else search_k
            similarities, indices = self._calculate_field_similarity(
                field, query_emb, search_k_adjusted
            )
            weight = weights.get(field, 0.0)
            
            # Filtrar e pontuar candidatos
            for idx, sim in zip(indices, similarities):
                # Aplicar filtro de UF
                if valid_indices_set is not None and idx not in valid_indices_set:
                    continue
                
                # Acumular scores
                if idx not in candidate_scores:
                    candidate_scores[idx] = 0.0
                    field_scores_map[idx] = {}
                
                candidate_scores[idx] += weight * sim
                field_scores_map[idx][field] = float(sim)
        
        # CEP matching
        if query.get('cep'):
            cep_weight = weights.get('cep', 0.0)
            for idx in candidate_scores.keys():
                db_cep = self.dataframe.iloc[idx]['cep']
                cep_score = self._calculate_cep_match(query.get('cep'), db_cep)
                candidate_scores[idx] += cep_weight * cep_score
                field_scores_map[idx]['cep'] = cep_score
        
        # Ordenar e retornar top_k
        sorted_candidates = sorted(
            candidate_scores.items(),
            key=lambda x: x[1],
            reverse=True
        )[:top_k]
        
        # Montar resposta
        results = []
        for idx, score in sorted_candidates:
            row = self.dataframe.iloc[idx]
            
            if score >= self.confidence_threshold:
                confidence = "high"
            elif score >= 0.6:
                confidence = "medium"
            else:
                confidence = "low"
            
            result = {
                "address": {
                    "logradouro": row['logradouro'],
                    "bairro": row['bairro'],
                    "cidade": row['cidade'],
                    "uf": row['uf'],
                    "cep": row['cep']
                },
                "score": float(score),
                "confidence": confidence,
                "field_scores": field_scores_map.get(idx, {})
            }
            results.append(result)
        
        response = {
            "results": results,
            "query": query,
            "total_found": len(results),
            "weights_used": weights
        }
        
        return json.dumps(response, ensure_ascii=False, indent=2)

## 4. Carregar DNE Real

In [ ]:
# Carregar DNE - apenas colunas necessárias
dne_path = Path('../data/dne.parquet')
print(f"📂 Carregando DNE de: {dne_path}")

# Selecionar apenas as colunas que precisamos (mais rápido e menos memória)
df_dne = pd.read_parquet(
    dne_path,
    columns=['logradouro_completo', 'bairro_completo', 'cidade_completo', 'uf', 'cep']
)

# Renomear colunas para formato esperado
df_dne = df_dne.rename(columns={
    'logradouro_completo': 'logradouro',
    'bairro_completo': 'bairro',
    'cidade_completo': 'cidade'
})

print(f"\n✅ Dataset carregado: {len(df_dne):,} registros")
print(f"📋 Colunas: {list(df_dne.columns)}")
print(f"\n📊 Distribuição por UF:")
print(df_dne['uf'].value_counts().head(10))

## 5. Construir Índices (EXECUTAR UMA VEZ) - ~5-10min na T4

In [ ]:
# Inicializar serviço de embeddings com FP16 (T4 optimization)
embedding_service = EmbeddingServiceGPU(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    use_fp16=True  # 2x speedup na T4
)

In [ ]:
# Construir índices HNSW
# use_gpu_index=False: índice fica na CPU (economiza VRAM para embeddings)
# use_gpu_index=True: índice na GPU (busca 5-10x mais rápida, mas consome VRAM)
index_builder = IndexBuilderGPU(embedding_service, use_gpu_index=False)

# Parâmetros balanceados para 1.5M registros:
# M=32: ~99.5% recall
# efSearch=32: balanceado entre velocidade e precisão
indices = index_builder.build_indices(
    df_dne,
    use_hnsw=True,
    M=32,
    efSearch=32
)

In [ ]:
# SALVAR índices para não precisar reconstruir
index_builder.save_indices('../data/indices_gpu_t4')

## 6. Carregar Índices (RÁPIDO - use sempre) - ~5s

In [ ]:
# Carregar modelo
embedding_service = EmbeddingServiceGPU(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    use_fp16=True
)

# Carregar índices salvos
index_builder = IndexBuilderGPU(embedding_service, use_gpu_index=False)
indices, df_dne = index_builder.load_indices('../data/indices_gpu_t4')

# Inicializar motor de busca OTIMIZADO (com filtro de UF inteligente)
search_engine = SearchEngineOptimized(embedding_service, indices, df_dne)
print(f"🚀 Sistema pronto! ({len(df_dne):,} endereços indexados)")

## 6.1. Teste Rápido - Comparação Antes/Depois

In [ ]:
# Testar com o endereço que estava retornando apenas 70%
query_test = {
    "logradouro": "avenida rio branco",
    "bairro": "cidade alta",
    "cidade": "natal",
    "uf": "RN",
    "cep": "59025000"
}

print("🎯 Teste com SearchEngineOptimized")
print("="*60)
print(f"Query: {query_test}\n")

start = time.time()
result_json = search_engine.search(query_test, top_k=3, search_k=100)
elapsed = time.time() - start
result = json.loads(result_json)

print(f"\n⚡ Busca concluída em: {elapsed*1000:.1f}ms\n")
print("✅ Top 3 Resultados:\n")

for i, res in enumerate(result['results'], 1):
    addr = res['address']
    print(f"{i}. Score: {res['score']:.4f} ({res['score']*100:.2f}%) - Confidence: {res['confidence']}")
    print(f"   {addr['logradouro']}")
    print(f"   {addr['bairro']} - {addr['cidade']}/{addr['uf']}")
    print(f"   CEP: {addr['cep']}")
    print(f"   Field scores: ", end="")
    for field, score in res['field_scores'].items():
        print(f"{field}={score:.3f} ", end="")
    print("\n")

# Verificação
top_score = result['results'][0]['score']
if top_score >= 0.98:
    print(f"🎉 SUCESSO! Score corrigido: {top_score:.4f} (≥98%)")
    print(f"✅ Problema de candidatos diferentes RESOLVIDO!")
else:
    print(f"⚠️  Score obtido: {top_score:.4f}")
    print(f"   (Esperado ≥0.98 para match perfeito)")

## 7. Teste de Busca

In [ ]:
# Exemplo de busca
query = {
    'logradouro': 'Rua das Flores',
    'bairro': 'Centro',
    'cidade': 'São Paulo',
    'uf': 'SP',
    'cep': '01000-000'
}

print("⏱️  Testando performance da busca...\n")

# Medir tempo
start = time.time()
result = search_engine.search(query, top_k=5)
elapsed = time.time() - start

print(f"⚡ Busca concluída em: {elapsed*1000:.1f}ms")
print(f"\nResultados:")
print(result)

## 7.1. Teste com Endereço Real (Match Perfeito)

In [ ]:
# Testar com o endereço exato do exemplo (Natal/RN)
query_test = {
    "logradouro": "avenida rio branco",
    "bairro": "cidade alta",
    "cidade": "natal",
    "uf": "RN",
    "cep": "59025000"
}

print("🎯 Testando match perfeito com endereço real da base...\n")
print("📍 Query:")
print(f"   {query_test['logradouro']}")
print(f"   {query_test['bairro']} - {query_test['cidade']}/{query_test['uf']}")
print(f"   CEP: {query_test['cep']}\n")

# Buscar
start = time.time()
result_json = search_engine.search(query_test, top_k=3)
elapsed = time.time() - start
result = json.loads(result_json)

# Mostrar resultado
print(f"⚡ Busca concluída em: {elapsed*1000:.1f}ms\n")
print("✅ Top 3 Resultados:\n")

for i, res in enumerate(result['results'], 1):
    addr = res['address']
    print(f"{i}. Score: {res['score']:.4f} ({res['score']*100:.2f}%) - Confidence: {res['confidence']}")
    print(f"   {addr['logradouro']}")
    print(f"   {addr['bairro']} - {addr['cidade']}/{addr['uf']}")
    print(f"   CEP: {addr['cep']}")
    print(f"   Field scores: ", end="")
    for field, score in res['field_scores'].items():
        print(f"{field}={score:.3f} ", end="")
    print("\n")

# Verificação
top_score = result['results'][0]['score']
if top_score >= 0.98:
    print(f"🎉 SUCESSO! Score correto: {top_score:.4f} (≥98%)")
else:
    print(f"⚠️  Score esperado ≥0.98, obtido: {top_score:.4f}")
    print(f"   Pesos usados: {result['weights_used']}")

## 7.2. Diagnóstico: Por que não bate 100%?

In [ ]:
# Diagnóstico detalhado do problema de score
print("🔍 DIAGNÓSTICO: Por que o score não chega a 100%?\n")
print("="*60)

# 1. Verificar se o endereço exato existe na base
query_diagnostic = {
    "logradouro": "avenida rio branco",
    "bairro": "cidade alta",
    "cidade": "natal",
    "uf": "RN",
    "cep": "59025000"
}

# Procurar correspondência EXATA no dataframe
print("\n1️⃣ Verificando se endereço EXATO existe na base:")
print("-"*60)

# Normalizar os termos de busca
from unidecode import unidecode

def normalize_for_search(text):
    if pd.isna(text):
        return ""
    return unidecode(str(text).lower().strip())

# Filtrar por UF primeiro
df_rn = df_dne[df_dne['uf'] == 'RN'].copy()
print(f"   Total de endereços no RN: {len(df_rn):,}")

# Buscar por CEP
cep_matches = df_rn[df_rn['cep'] == query_diagnostic['cep']]
print(f"\n   Endereços com CEP {query_diagnostic['cep']}: {len(cep_matches)}")

if len(cep_matches) > 0:
    print("\n   Primeiros 3 registros com este CEP:")
    for idx, row in cep_matches.head(3).iterrows():
        print(f"\n   [{idx}]")
        print(f"      Logradouro: {row['logradouro']}")
        print(f"      Bairro: {row['bairro']}")
        print(f"      Cidade: {row['cidade']}")
        print(f"      CEP: {row['cep']}")

# Buscar match exato nos textos normalizados
print("\n2️⃣ Verificando normalização dos campos:")
print("-"*60)

query_norm = {
    'logradouro': normalize_for_search(query_diagnostic['logradouro']),
    'bairro': normalize_for_search(query_diagnostic['bairro']),
    'cidade': normalize_for_search(query_diagnostic['cidade'])
}

print(f"   Query normalizada:")
print(f"      Logradouro: '{query_norm['logradouro']}'")
print(f"      Bairro: '{query_norm['bairro']}'")
print(f"      Cidade: '{query_norm['cidade']}'")

# Verificar se existe match exato
if len(cep_matches) > 0:
    first_match = cep_matches.iloc[0]
    db_norm = {
        'logradouro': normalize_for_search(first_match['logradouro']),
        'bairro': normalize_for_search(first_match['bairro']),
        'cidade': normalize_for_search(first_match['cidade'])
    }
    
    print(f"\n   Primeiro registro da base normalizado:")
    print(f"      Logradouro: '{db_norm['logradouro']}'")
    print(f"      Bairro: '{db_norm['bairro']}'")
    print(f"      Cidade: '{db_norm['cidade']}'")
    
    print(f"\n   Comparação:")
    for field in ['logradouro', 'bairro', 'cidade']:
        match = "✅ MATCH" if query_norm[field] == db_norm[field] else f"❌ DIFERENTE"
        print(f"      {field}: {match}")

# 3. Testar busca campo por campo
print("\n3️⃣ Testando similaridade por campo individual:")
print("-"*60)

for field in ['logradouro', 'bairro', 'cidade']:
    query_text = query_diagnostic[field]
    query_emb = search_engine.embedding_service.embed_text(query_text)
    
    # Buscar top 3
    similarities, indices = search_engine._calculate_field_similarity(field, query_emb, top_k=3)
    
    print(f"\n   Campo: {field}")
    print(f"   Query: '{query_text}'")
    print(f"   Top 3 resultados:")
    
    for i, (sim, idx) in enumerate(zip(similarities, indices), 1):
        db_value = df_dne.iloc[idx][field]
        db_uf = df_dne.iloc[idx]['uf']
        print(f"      {i}. [{idx}] Score: {sim:.4f} | UF: {db_uf} | Valor: '{db_value}'")

print("\n" + "="*60)
print("💡 CONCLUSÃO:")
print("="*60)
print("Se os campos estão batendo mas o score final é baixo,")
print("o problema é que a busca vetorial está retornando")
print("CANDIDATOS DIFERENTES para cada campo (logradouro, bairro, cidade).")
print("\nSolução: os candidatos precisam se sobrepor (mesmo índice do dataframe)")
print("para que todos os scores sejam somados no mesmo resultado final.")

In [ ]:
print("🔬 INVESTIGAÇÃO PROFUNDA: Análise dos Candidatos Retornados")
print("="*80)

query_analysis = {
    "logradouro": "avenida rio branco",
    "bairro": "cidade alta",
    "cidade": "natal",
    "uf": "RN",
    "cep": "59025000"
}

# 1. Testar cada campo individualmente e coletar os índices retornados
print("\n1️⃣ CANDIDATOS RETORNADOS POR CAMPO (Top 10)")
print("-"*80)

field_candidates = {}
weights = search_engine._get_dynamic_weights(query_analysis)

for field in ['logradouro', 'bairro', 'cidade', 'cep']:
    print(f"\n📌 Campo: {field.upper()} (peso: {weights.get(field, 0.0):.2f})")
    print("-"*80)
    
    if field == 'cep':
        # CEP é matching exato, não vetorial
        cep_query = query_analysis['cep']
        df_filtered = df_dne[df_dne['uf'] == 'RN']
        cep_matches = df_filtered[df_filtered['cep'] == cep_query]
        
        print(f"   Matches exatos de CEP: {len(cep_matches)}")
        
        if len(cep_matches) > 0:
            field_candidates['cep'] = cep_matches.index.tolist()[:10]
            print(f"\n   Top 10 índices com CEP {cep_query}:")
            for i, idx in enumerate(field_candidates['cep'][:10], 1):
                row = df_dne.iloc[idx]
                print(f"   {i}. [idx={idx:6d}] {row['logradouro'][:50]:50s} - {row['bairro'][:20]:20s}")
        else:
            field_candidates['cep'] = []
            print(f"   ⚠️ Nenhum match encontrado!")
        
    else:
        # Busca vetorial
        query_text = query_analysis[field]
        query_emb = search_engine.embedding_service.embed_text(query_text)
        
        # Buscar top 100 candidatos (mesmo search_k da busca real)
        similarities, indices = search_engine._calculate_field_similarity(field, query_emb, top_k=100)
        
        # Filtrar por UF
        valid_indices = []
        valid_sims = []
        for sim, idx in zip(similarities, indices):
            if df_dne.iloc[idx]['uf'] == 'RN':
                valid_indices.append(idx)
                valid_sims.append(sim)
                if len(valid_indices) >= 10:
                    break
        
        field_candidates[field] = valid_indices
        
        print(f"   Query: '{query_text}'")
        print(f"   Top 10 candidatos no RN:")
        for i, (idx, sim) in enumerate(zip(valid_indices[:10], valid_sims[:10]), 1):
            row = df_dne.iloc[idx]
            print(f"   {i}. [idx={idx:6d}] Score: {sim:.4f} | '{row[field]}'")

# 2. Analisar sobreposição entre os conjuntos
print("\n\n2️⃣ ANÁLISE DE SOBREPOSIÇÃO DE CANDIDATOS")
print("="*80)

# Converter para sets
sets = {field: set(indices) for field, indices in field_candidates.items()}

print(f"\nTotal de candidatos únicos por campo:")
for field, indices_set in sets.items():
    print(f"   {field:12s}: {len(indices_set):3d} candidatos")

# Interseção entre todos os campos
all_fields = ['logradouro', 'bairro', 'cidade', 'cep']
intersection = sets[all_fields[0]]
for field in all_fields[1:]:
    intersection = intersection.intersection(sets[field])

print(f"\n📊 Candidatos que aparecem em TODOS os campos: {len(intersection)}")

if len(intersection) > 0:
    print(f"\n✅ Índices na interseção de todos os campos:")
    for idx in sorted(list(intersection))[:5]:
        row = df_dne.iloc[idx]
        print(f"\n   [idx={idx}]")
        print(f"      Logradouro: {row['logradouro']}")
        print(f"      Bairro: {row['bairro']}")
        print(f"      Cidade: {row['cidade']}")
        print(f"      CEP: {row['cep']}")
else:
    print(f"\n❌ PROBLEMA IDENTIFICADO: Nenhum candidato aparece em todos os campos!")
    print(f"\n   Analisando interseções parciais:")
    
    # Testar combinações 3 a 3
    from itertools import combinations
    
    for combo in combinations(all_fields, 3):
        combo_set = sets[combo[0]]
        for field in combo[1:]:
            combo_set = combo_set.intersection(sets[field])
        
        if len(combo_set) > 0:
            print(f"\n   ✓ {' + '.join(combo):40s}: {len(combo_set):2d} candidatos em comum")
            
            # Mostrar qual campo está faltando
            missing_field = [f for f in all_fields if f not in combo][0]
            print(f"     (falta: {missing_field})")
            
            # Mostrar primeiro candidato dessa interseção
            first_idx = sorted(list(combo_set))[0]
            row = df_dne.iloc[first_idx]
            print(f"     Exemplo [idx={first_idx}]: {row['logradouro'][:40]} - {row['bairro'][:20]}")

# 3. Simular o algoritmo de busca manualmente
print("\n\n3️⃣ SIMULAÇÃO DO ALGORITMO DE BUSCA")
print("="*80)

# Coletar top 5 candidatos por campo
top5_per_field = {}
for field in ['logradouro', 'bairro', 'cidade']:
    query_text = query_analysis[field]
    query_emb = search_engine.embedding_service.embed_text(query_text)
    similarities, indices = search_engine._calculate_field_similarity(field, query_emb, top_k=100)
    
    # Filtrar RN e pegar top 5
    top5 = []
    for sim, idx in zip(similarities, indices):
        if df_dne.iloc[idx]['uf'] == 'RN':
            top5.append((idx, sim))
            if len(top5) >= 5:
                break
    
    top5_per_field[field] = top5

# Simular acumulação de scores
print("\nSimulando acumulação de scores (top 5 de cada campo):")
print("-"*80)

candidate_scores = {}
field_scores_map = {}

for field in ['logradouro', 'bairro', 'cidade']:
    weight = weights[field]
    print(f"\n{field} (peso={weight:.2f}):")
    
    for idx, sim in top5_per_field[field]:
        if idx not in candidate_scores:
            candidate_scores[idx] = 0.0
            field_scores_map[idx] = {}
        
        candidate_scores[idx] += weight * sim
        field_scores_map[idx][field] = sim
        
        print(f"   [idx={idx}] sim={sim:.4f} → score parcial={candidate_scores[idx]:.4f}")

# Adicionar CEP
print(f"\nCEP (peso={weights['cep']:.2f}):")
for idx in candidate_scores.keys():
    db_cep = df_dne.iloc[idx]['cep']
    cep_score = search_engine._calculate_cep_match(query_analysis['cep'], db_cep)
    candidate_scores[idx] += weights['cep'] * cep_score
    field_scores_map[idx]['cep'] = cep_score
    
    if cep_score > 0:
        print(f"   [idx={idx}] cep_match={cep_score:.4f} → score final={candidate_scores[idx]:.4f}")

# Ordenar e mostrar top 5
print("\n\n🏆 RANKING FINAL:")
print("="*80)
sorted_candidates = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)[:5]

for rank, (idx, score) in enumerate(sorted_candidates, 1):
    row = df_dne.iloc[idx]
    scores = field_scores_map[idx]
    
    print(f"\n{rank}. Score: {score:.4f} ({score*100:.2f}%) [idx={idx}]")
    print(f"   {row['logradouro']}")
    print(f"   {row['bairro']} - {row['cidade']}/{row['uf']}")
    print(f"   CEP: {row['cep']}")
    print(f"   Scores: ", end="")
    for field in ['logradouro', 'bairro', 'cidade', 'cep']:
        if field in scores:
            print(f"{field}={scores[field]:.3f} ", end="")
    print()

print("\n" + "="*80)
print("💡 CONCLUSÃO DA INVESTIGAÇÃO:")
print("="*80)

top_score = sorted_candidates[0][1]
top_idx = sorted_candidates[0][0]
top_field_scores = field_scores_map[top_idx]

if top_score >= 0.98:
    print("✅ Score ≥98%: Sistema funcionando perfeitamente!")
elif top_score >= 0.90:
    print("⚠️  Score entre 90-98%: Candidatos não se sobrepõem completamente")
    print("\nAnálise:")
    
    # Verificar quais campos estão perfeitos
    perfect_fields = [f for f, s in top_field_scores.items() if s >= 0.99]
    imperfect_fields = [f for f, s in top_field_scores.items() if s < 0.99]
    
    print(f"   Campos com match perfeito: {', '.join(perfect_fields) if perfect_fields else 'nenhum'}")
    print(f"   Campos com match imperfeito: {', '.join(imperfect_fields) if imperfect_fields else 'nenhum'}")
    
    # Calcular qual seria o score máximo possível
    max_possible = sum(weights[f] for f in perfect_fields if f in weights)
    print(f"\n   Score máximo possível com campos perfeitos: {max_possible:.4f} ({max_possible*100:.2f}%)")
    print(f"   Score obtido: {top_score:.4f} ({top_score*100:.2f}%)")
    print(f"   Diferença: {(max_possible - top_score)*100:.2f}%")
    
    if len(intersection) == 0:
        print(f"\n   🔍 CAUSA RAIZ: Os candidatos de cada campo são DIFERENTES")
        print(f"      Nenhum índice do DataFrame aparece em todos os campos simultaneamente.")
        print(f"      Por isso, mesmo com todos os campos tendo score 1.0 individualmente,")
        print(f"      esses scores perfeitos estão em registros DIFERENTES.")
else:
    print("❌ Score <90%: Problema de qualidade na busca vetorial")

print("="*80)

## 7.3. Investigação Profunda: Analisando os Candidatos Retornados

## 7.4. Estratégia MELHOR: IDSelector do FAISS para Filtragem Precisa

In [ ]:
"""
🎯 ESTRATÉGIA MELHORADA: Filtragem Precisa por UF

Problema atual:
- search_k * 10 = impreciso e lento
- Busca global depois filtra = desperdício

Soluções possíveis:

1. **IDSelector do FAISS** (RECOMENDADO) ✅
   - Filtra DURANTE a busca vetorial
   - Mantém search_k original (preciso)
   - Suportado por IndexHNSWFlat
   - Zero overhead de memória

2. **Índices separados por UF**
   - 27 índices (um por UF)
   - Busca direto no índice correto
   - Melhor recall, mas 27x mais memória

3. **IndexIDMap + IDSelector**
   - Wrapper que adiciona IDs customizados
   - Permite filtrar por qualquer critério
   - Overhead mínimo

Vamos implementar a Opção 1 (IDSelector):
"""

class SearchEngineWithIDSelector(SearchEngine):
    """
    Motor de busca com filtragem PRECISA usando IDSelector do FAISS.
    
    IDSelector filtra os candidatos DURANTE a busca vetorial, não depois.
    Isso mantém o search_k original e garante precisão.
    """
    
    def __init__(
        self,
        embedding_service: EmbeddingServiceGPU,
        indices: Dict[str, faiss.Index],
        dataframe: pd.DataFrame
    ):
        super().__init__(embedding_service, indices, dataframe)
        
        # Criar mapeamento de índices por UF
        print("🗺️  Criando mapeamento de índices por UF para IDSelector...")
        self.uf_to_indices = {}
        for uf in self.dataframe['uf'].unique():
            uf_mask = self.dataframe['uf'] == uf
            # IDSelector precisa de numpy array de int64
            self.uf_to_indices[uf] = np.where(uf_mask)[0].astype(np.int64)
        
        total_ufs = len(self.uf_to_indices)
        print(f"✅ Mapeamento criado: {total_ufs} UFs indexadas")
    
    def _calculate_field_similarity(
        self,
        field: str,
        query_embedding: np.ndarray,
        top_k: int = 100
    ) -> tuple:
        """Busca vetorial com conversão correta de distância para similaridade"""
        index = self.indices[field]
        query_embedding = query_embedding.reshape(1, -1).astype(np.float32)
        distances, indices = index.search(query_embedding, top_k)
        
        # Converter distâncias L2² para similaridade cosseno
        similarities = np.clip(1.0 - (distances[0] / 2.0), 0.0, 1.0)
        
        # Considerar distâncias muito pequenas como match perfeito
        epsilon = 1e-6
        similarities[distances[0] < epsilon] = 1.0
        
        return similarities, indices[0]
    
    def _calculate_field_similarity_with_filter(
        self,
        field: str,
        query_embedding: np.ndarray,
        uf: str,
        top_k: int = 100
    ) -> tuple:
        """
        Busca vetorial com filtragem por UF usando IDSelector do FAISS.
        
        IDSelector filtra DURANTE a busca, mantendo a precisão do HNSW.
        """
        index = self.indices[field]
        query_embedding = query_embedding.reshape(1, -1).astype(np.float32)
        
        # Criar IDSelector para o UF
        valid_ids = self.uf_to_indices[uf]
        
        # IDSelectorBatch: aceita um array de IDs válidos
        selector = faiss.IDSelectorBatch(valid_ids)
        
        # SearchParameters com IDSelector
        params = faiss.SearchParametersHNSW()
        params.sel = selector
        
        # Busca com filtragem
        distances, indices = index.search(query_embedding, top_k, params=params)
        
        # Converter distâncias para similaridades
        similarities = np.clip(1.0 - (distances[0] / 2.0), 0.0, 1.0)
        epsilon = 1e-6
        similarities[distances[0] < epsilon] = 1.0
        
        return similarities, indices[0]
    
    def search(
        self,
        query: Dict[str, str],
        top_k: int = 5,
        search_k: int = 100
    ) -> str:
        """
        Busca com filtragem PRECISA por UF usando IDSelector.
        
        Vantagens:
        - Filtra DURANTE a busca vetorial (não depois)
        - Mantém search_k original (sem multiplicar por 10)
        - Recall perfeito dentro do UF
        - Performance similar à busca global
        """
        weights = self._get_dynamic_weights(query)
        query_embeddings = self.embedding_service.embed_address_fields(query)
        
        query_uf = query.get('uf')
        use_filter = query_uf and query_uf in self.uf_to_indices
        
        if use_filter:
            print(f"🎯 Filtrando por UF={query_uf} com IDSelector (preciso)")
        
        candidate_scores = {}
        field_scores_map = {}
        
        # Busca vetorial por campo
        for field in ['logradouro', 'bairro', 'cidade']:
            if not query.get(field):
                continue
            
            query_emb = query_embeddings[field]
            
            # Usar IDSelector se UF fornecido
            if use_filter:
                similarities, indices = self._calculate_field_similarity_with_filter(
                    field, query_emb, query_uf, search_k
                )
            else:
                similarities, indices = self._calculate_field_similarity(
                    field, query_emb, search_k
                )
            
            weight = weights.get(field, 0.0)
            
            # Acumular scores (sem filtro adicional - já filtrado pelo IDSelector)
            for idx, sim in zip(indices, similarities):
                if idx == -1:  # FAISS retorna -1 para slots vazios
                    continue
                
                if idx not in candidate_scores:
                    candidate_scores[idx] = 0.0
                    field_scores_map[idx] = {}
                
                candidate_scores[idx] += weight * sim
                field_scores_map[idx][field] = float(sim)
        
        # CEP matching
        if query.get('cep'):
            cep_weight = weights.get('cep', 0.0)
            for idx in candidate_scores.keys():
                db_cep = self.dataframe.iloc[idx]['cep']
                cep_score = self._calculate_cep_match(query.get('cep'), db_cep)
                candidate_scores[idx] += cep_weight * cep_score
                field_scores_map[idx]['cep'] = cep_score
        
        # Ordenar e retornar top_k
        sorted_candidates = sorted(
            candidate_scores.items(),
            key=lambda x: x[1],
            reverse=True
        )[:top_k]
        
        # Montar resposta
        results = []
        for idx, score in sorted_candidates:
            row = self.dataframe.iloc[idx]
            
            if score >= self.confidence_threshold:
                confidence = "high"
            elif score >= 0.6:
                confidence = "medium"
            else:
                confidence = "low"
            
            result = {
                "address": {
                    "logradouro": row['logradouro'],
                    "bairro": row['bairro'],
                    "cidade": row['cidade'],
                    "uf": row['uf'],
                    "cep": row['cep']
                },
                "score": float(score),
                "confidence": confidence,
                "field_scores": field_scores_map.get(idx, {})
            }
            results.append(result)
        
        response = {
            "results": results,
            "query": query,
            "total_found": len(results),
            "weights_used": weights
        }
        
        return json.dumps(response, ensure_ascii=False, indent=2)

print("✅ SearchEngineWithIDSelector criado!")
print("\n📊 Comparação de estratégias:")
print("-"*60)
print("SearchEngine (original):")
print("  - Busca global, filtra depois")
print("  - Rápido mas pode perder candidatos")
print("")
print("SearchEngineOptimized (atual):")
print("  - search_k * 10 para compensar filtro")
print("  - Funciona mas impreciso e lento")
print("")
print("SearchEngineWithIDSelector (NOVO): ✅")
print("  - IDSelector filtra DURANTE busca")
print("  - Mantém search_k original")
print("  - Precisão perfeita + performance")
print("-"*60)

### 7.4.1. Teste: SearchEngineWithIDSelector

In [ ]:
# Criar novo search engine com IDSelector
search_engine_idselector = SearchEngineWithIDSelector(embedding_service, indices, df_dne)

# Testar com o mesmo endereço problemático
query_test = {
    "logradouro": "avenida rio branco",
    "bairro": "cidade alta",
    "cidade": "natal",
    "uf": "RN",
    "cep": "59025000"
}

print("\n🎯 Teste com SearchEngineWithIDSelector (IDSelector do FAISS)")
print("="*70)
print(f"Query: {query_test}\n")

start = time.time()
result_json = search_engine_idselector.search(query_test, top_k=3, search_k=100)
elapsed = time.time() - start
result = json.loads(result_json)

print(f"\n⚡ Busca concluída em: {elapsed*1000:.1f}ms\n")
print("✅ Top 3 Resultados:\n")

for i, res in enumerate(result['results'], 1):
    addr = res['address']
    print(f"{i}. Score: {res['score']:.4f} ({res['score']*100:.2f}%) - {res['confidence']}")
    print(f"   {addr['logradouro']}")
    print(f"   {addr['bairro']} - {addr['cidade']}/{addr['uf']}")
    print(f"   CEP: {addr['cep']}")
    print(f"   Field scores: ", end="")
    for field, score in res['field_scores'].items():
        print(f"{field}={score:.3f} ", end="")
    print("\n")

# Análise
top_score = result['results'][0]['score']
print("="*70)
print("📊 ANÁLISE DO RESULTADO:")
print("="*70)

if top_score >= 0.98:
    print(f"🎉 EXCELENTE! Score: {top_score:.4f} (≥98%)")
    print(f"✅ IDSelector funcionou perfeitamente!")
    print(f"✅ Filtragem precisa durante busca vetorial")
elif top_score >= 0.90:
    print(f"✅ BOM! Score: {top_score:.4f} (90-98%)")
    print(f"⚠️  Ainda há pequena diferença")
    
    # Verificar quais campos contribuíram
    top_fields = result['results'][0]['field_scores']
    print(f"\nContribuição por campo:")
    weights = result['weights_used']
    for field in ['logradouro', 'bairro', 'cidade', 'cep']:
        if field in top_fields:
            field_score = top_fields[field]
            weight = weights.get(field, 0.0)
            contribution = field_score * weight
            print(f"  {field:12s}: {field_score:.3f} × {weight:.2f} = {contribution:.3f}")
        else:
            print(f"  {field:12s}: (não encontrado)")
else:
    print(f"❌ Score baixo: {top_score:.4f}")
    print(f"   Problema persiste mesmo com IDSelector")

print("="*70)

### 7.4.2. Análise Detalhada: IDSelector vs Filtro Posterior

In [ ]:
print("🔬 ANÁLISE COMPARATIVA: IDSelector vs Filtro Posterior")
print("="*80)

query_comparison = {
    "logradouro": "avenida rio branco",
    "bairro": "cidade alta", 
    "cidade": "natal",
    "uf": "RN",
    "cep": "59025000"
}

# Testar campo por campo
for field in ['logradouro', 'bairro', 'cidade']:
    print(f"\n📌 Campo: {field.upper()}")
    print("-"*80)
    
    query_emb = search_engine_idselector.embedding_service.embed_text(query_comparison[field])
    
    # 1. Busca SEM filtro (global)
    print(f"\n1️⃣ Busca GLOBAL (sem filtro):")
    sims_global, idxs_global = search_engine_idselector._calculate_field_similarity(
        field, query_emb, top_k=10
    )
    
    rn_count = sum(1 for idx in idxs_global if df_dne.iloc[idx]['uf'] == 'RN')
    print(f"   Total retornados: 10")
    print(f"   Do RN: {rn_count}")
    print(f"   Top 3:")
    for i, (idx, sim) in enumerate(zip(idxs_global[:3], sims_global[:3]), 1):
        uf = df_dne.iloc[idx]['uf']
        value = df_dne.iloc[idx][field]
        print(f"      {i}. [idx={idx}] UF={uf} | sim={sim:.4f} | '{value[:40]}'")
    
    # 2. Busca COM IDSelector
    print(f"\n2️⃣ Busca COM IDSelector (filtro durante busca):")
    sims_filtered, idxs_filtered = search_engine_idselector._calculate_field_similarity_with_filter(
        field, query_emb, 'RN', top_k=10
    )
    
    # Remover índices -1 (slots vazios)
    valid_mask = idxs_filtered != -1
    idxs_filtered = idxs_filtered[valid_mask]
    sims_filtered = sims_filtered[valid_mask]
    
    print(f"   Total retornados: {len(idxs_filtered)}")
    print(f"   Todos do RN: ✅")
    print(f"   Top 3:")
    for i, (idx, sim) in enumerate(zip(idxs_filtered[:3], sims_filtered[:3]), 1):
        uf = df_dne.iloc[idx]['uf']
        value = df_dne.iloc[idx][field]
        print(f"      {i}. [idx={idx}] UF={uf} | sim={sim:.4f} | '{value[:40]}'")
    
    # 3. Verificar se o índice 955197 (do CEP correto) aparece
    target_idx = 955197
    print(f"\n3️⃣ Verificação: idx={target_idx} (CEP correto) aparece?")
    
    if target_idx in idxs_global:
        pos = np.where(idxs_global == target_idx)[0][0]
        print(f"   Busca global: SIM (posição {pos+1}, sim={sims_global[pos]:.4f})")
    else:
        print(f"   Busca global: NÃO (fora do top 10)")
    
    if target_idx in idxs_filtered:
        pos = np.where(idxs_filtered == target_idx)[0][0]
        print(f"   Com IDSelector: SIM (posição {pos+1}, sim={sims_filtered[pos]:.4f})")
    else:
        print(f"   Com IDSelector: NÃO (fora do top 10)")

print("\n" + "="*80)
print("💡 CONCLUSÃO:")
print("="*80)
print("IDSelector garante que TODOS os resultados sejam do UF correto,")
print("aumentando a chance de encontrar o candidato certo (idx=955197)")
print("que tem CEP, logradouro corretos mas pode estar fora do top global.")
print("="*80)

## 8. Benchmark de Performance

In [ ]:
# Testar múltiplas buscas
n_searches = 50
times = []

print(f"🔥 Executando {n_searches} buscas para benchmark...\n")

for i in tqdm(range(n_searches), desc="Buscas"):
    sample = df_dne.sample(1).iloc[0]
    query = {
        'logradouro': sample['logradouro'],
        'bairro': sample['bairro'],
        'cidade': sample['cidade'],
        'uf': sample['uf']
    }
    
    start = time.time()
    result = search_engine.search(query, top_k=5)
    elapsed = time.time() - start
    times.append(elapsed)

times_ms = [t * 1000 for t in times]

print(f"\n{'='*60}")
print(f"📊 ESTATÍSTICAS DE PERFORMANCE")
print(f"{'='*60}")
print(f"Média:    {np.mean(times_ms):.1f}ms")
print(f"Mediana:  {np.median(times_ms):.1f}ms")
print(f"Min:      {np.min(times_ms):.1f}ms")
print(f"Max:      {np.max(times_ms):.1f}ms")
print(f"P95:      {np.percentile(times_ms, 95):.1f}ms")
print(f"P99:      {np.percentile(times_ms, 99):.1f}ms")
print(f"\n⚡ Throughput: ~{1000/np.mean(times_ms):.0f} queries/segundo")
print(f"{'='*60}")

## 📝 Notas de Otimização para g4dn.2xlarge

### Performance esperada na T4:
| Operação | Tempo |
|----------|-------|
| Construção inicial (1.5M) | 5-10min |
| Carregamento índices | <10s |
| Busca (p50) | 30-50ms |
| Busca (p95) | 60-80ms |
| Throughput | ~500-1000 q/s |

### Ajustes finos:

**1. Modelo de embedding:**
```python
# RÁPIDO (atual)
"paraphrase-multilingual-MiniLM-L12-v2"  # 384 dims, batch 256

# MUITO RÁPIDO
"all-MiniLM-L6-v2"  # 384 dims, batch 256, inglês mas OK

# PRECISO (mais lento)
"paraphrase-multilingual-mpnet-base-v2"  # 768 dims, batch 128
```

**2. Parâmetros HNSW:**
```python
# Mais rápido (98% recall)
M=16, efSearch=16

# Balanceado (99.5% recall) ✅
M=32, efSearch=32

# Mais preciso (99.9% recall)
M=64, efSearch=64
```

**3. FAISS GPU (opcional):**
```bash
# Instalar faiss-gpu
pip uninstall faiss-cpu
pip install faiss-gpu
```
```python
# Usar índices na GPU (busca 5-10x mais rápida)
index_builder = IndexBuilderGPU(embedding_service, use_gpu_index=True)
```
⚠️ **Atenção**: Índices na GPU consomem VRAM (pode conflitar com embeddings)

### Custos AWS:
- **g4dn.2xlarge On-Demand**: ~$0.75/hora
- **g4dn.2xlarge Spot**: ~$0.25/hora (70% desconto)
- **Construção única**: $0.05 (5min spot)
- **Alternativa**: g4dn.xlarge ($0.53/h) se não precisa de 32GB RAM

### Produção:
1. **Construir índices uma vez** (5-10min)
2. **Salvar em S3** ou volume persistente
3. **Carregar na inicialização** (<10s)
4. **API com FastAPI** + autoscaling
5. **Monitoramento**: CloudWatch + logs de latência